In [1]:
import os
import pandas as pd
import re
import subprocess
genome = os.path.expanduser("~/clement/genomes/Homo_sapiens/UCSC/hg38/Sequence/WholeGenomeFasta/genome.fa")

In [2]:
#CHANGE-seq reveals genetic and epigenetic effects on CRISPR–Cas9 genome-wide activity
#Cicera R. Lazzarotto, Nikolay L. Malinin, Yichao Li, Ruochi Zhang, Yang Yang, GaHyun Lee, Eleanor Cowley, Yanghua He, Xin Lan, Kasey Jividen, Varun Katta, Natalia G. Kolmakova, Christopher T. Petersen, Qian Qi, Evgheni Strelcov, Samantha Maragh, Giedre Krenciute, Jian Ma, Yong Cheng & Shengdar Q. Tsai 

In [3]:
#CTLA4_Site9: GGACTGAGGGCCATGGACACNGG

#Note that some of the targets are annotated with '|3|' in the supp_2 file, but this includes targets with 2 mismatches in the guide and 1 in the pam
#Also, there are some with '|2|' annotations that have one mismatch and a gap
#Here we'll only include targets with 2 mismatches and no gaps

In [4]:
targets_written = 0
with open('../supp_2.txt','r') as fin, open('01_make_demo.ipynb.genome.fa','w') as fout, open('01_make_demo.ipynb.genome.fa.info','w') as finfo:
    supp_line_head = fin.readline()
    for line in fin: #CTLA4_site_9|chr16:88695984-88696006:+|GGCCTGAGGCCCTTGGACACTGG|4040|3|A GGAGGAGGCAGGAGGCAGTGACT gccaccTCAGCACCTCCTGGATT
        if not line.strip():  # Skip empty lines
            continue
        line_els = line.split("\t")
        target_els = line_els[0].split("|")
        chrom, target_range, strand = target_els[1].split(":")
        num_mismatches = target_els[4]
        if int(num_mismatches) > 2:
            continue
        target_seq = target_els[2]
        if '-' in target_seq:
            print('Skipping target with gap: ' + line.strip())
            continue
        if not chrom.startswith('chr'):
            raise Exception('Cannot parse location from line ' + line.strip())
        target_start, target_end = target_range.split("-")
        seq_lines = subprocess.check_output(['samtools', 'faidx', genome, f'{chrom}:{int(target_start)-500}-{int(target_end)+500}']).decode('utf-8')
        seq = seq_lines[seq_lines.index('\n')+1:].replace('\n', '').upper()
        fout.write(f'>CTLA4_site{targets_written}\n{seq}\n')
        finfo.write(f'CTLA4_site{targets_written}\t{chrom}:{target_start}-{target_end}:{strand}\t{num_mismatches}\t{line_els[0]}\n')
        print('wrote ' + line_els[0])
        targets_written += 1
print(f'Wrote {targets_written} targets to 01_make_demo.ipynb.genome.fa')

Skipping target with gap: CTLA4_site_9|chr10:112098238-112098259:-|GGACTGAGGGCT-TGGACACTGG|546|2|A	TCAGGCTTCAGCAGACCCAGACA	GAGCCCATGAAGCAGGGTCCCAG
wrote CTLA4_site_9|chr2:203870828-203870850:+|GGACTGAGGGCCATGGACACGGG|12382|0|both
wrote CTLA4_site_9|chr9:135268714-135268736:+|GGACAGAGGGCCCTGGACACAGG|10036|2|both
wrote CTLA4_site_9|chr5:101611404-101611426:+|GGAATGAGGCCCATGGACACTGG|5244|2|both
wrote CTLA4_site_9|chr9:121888845-121888867:+|GGACTGGGGGCCTTGGACACAGG|2284|2|both
Skipping target with gap: CTLA4_site_9|chr9:36900766-36900787:-|GGACTGGGGGC-ATGGACACAGG|162|2|C	GGGGACCCGGGAGATAATGGAGCC	CCAGCTCTGCTCCTCCCCATCGT
Skipping target with gap: CTLA4_site_9|chr22:20321436-20321457:-|GGACTGAGGGC-AGGGACACTGG|70|2|C	CTCACAGCCCCCACCCCACCTG	tTGGTCACTGTCACCTCCCCCGG
Skipping target with gap: CTLA4_site_9|chr22:18903849-18903870:+|GGACTGAGGGC-AGGGACACTGG|42|2|C	GACCTGGCAGACCCCGGAGCTG	TTGGTCACTGTCACCTCCCCCGG
Skipping target with gap: CTLA4_site_9|chr22:45142368-45142389:-|GGCCTGAGGGCCATGGA-ACAGG|12|

In [5]:
# link 01_make_genome.ipynb.genome.fa to test_genome.fa
if not os.path.exists('demo_genome.fa'):
    os.symlink('01_make_demo.ipynb.genome.fa', 'demo_genome.fa')


In [6]:
subprocess.call(['bowtie2-build', 'demo_genome.fa', 'demo_genome'])

Settings:
  Output files: "demo_genome.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  demo_genome.fa
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 1023
Using parameters --bmax 768 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 768 --dcv 1024
Constructing suffix-array element generator
Building Differ

Building a SMALL index
Renaming demo_genome.3.bt2.tmp to demo_genome.3.bt2
Renaming demo_genome.4.bt2.tmp to demo_genome.4.bt2
Renaming demo_genome.1.bt2.tmp to demo_genome.1.bt2
Renaming demo_genome.2.bt2.tmp to demo_genome.2.bt2
Renaming demo_genome.rev.1.bt2.tmp to demo_genome.rev.1.bt2
Renaming demo_genome.rev.2.bt2.tmp to demo_genome.rev.2.bt2


0

In [7]:
samples = pd.read_csv('../samples.small.txt',sep="\t")
sample_names = samples['Name'].tolist()
for sample_name in sample_names:
    sample_fastq = f'{sample_name}.fastq.gz'
    if os.path.exists(sample_fastq):
        os.remove(sample_fastq)
print(sample_names)

['Treated_1', 'Treated_2', 'Treated_3', 'Control_1', 'Control_2', 'Control_3']


In [8]:

with open("../CRISPRessoSea_output_on_samples.small.txt/aggregated_stats_all.txt",'r') as fin:
    header = fin.readline()
    for line in fin:
        line_els = line.split("\t")
        target_mismatches = 5
        match = re.search(r'_OB(\d+)_', line_els[1])
        if match is None:
            if '_ON_' in line_els[1]:
                target_mismatches = 0
        else:
            target_mismatches = int(match.group(1))
        if target_mismatches > 2:
            continue
        region_name = line_els[12]
        for sample_name in sample_names:
            fastq_loc = f'../CRISPRessoSea_output_on_samples.small.txt/CRISPResso_output/CRISPRessoPooled_on_{sample_name}/MAPPED_REGIONS/REGION_{region_name}.fastq.gz'
            if not os.path.exists(fastq_loc):
                raise Exception('Cannot find fastq file for sample ' + sample_name + ' at ' + fastq_loc)
            subprocess.call(f'cat {fastq_loc} >> {sample_name}.fastq.gz', shell=True)

for sample_name in sample_names:
    file_line_count = subprocess.check_output([f'zcat {sample_name}.fastq.gz | wc -l'], shell=True).decode('utf-8').strip()
    print(f'File {sample_name}.fastq.gz has {file_line_count} lines')


File Treated_1.fastq.gz has 4488 lines
File Treated_2.fastq.gz has 4100 lines
File Treated_3.fastq.gz has 4364 lines
File Control_1.fastq.gz has 4360 lines
File Control_2.fastq.gz has 4440 lines
File Control_3.fastq.gz has 4464 lines


In [9]:
with open('samples.demo.txt','w') as fout:
    fout.write('Name\tr1\n')
    for sample_name in sample_names:
        fout.write(f'{sample_name}\t{sample_name}.fastq.gz\n')